In [78]:
import pandas as pd
import os
from pandas import DataFrame as df

In [97]:
# import the two csv files needed
#STEP1 : Import the csv file containing the admin boundaries in level 2 and the production in ha per district
path = r"C:\Oluchi\Benin\SNV DATA"
name_of_file = "benin_cassava_production.csv"
# Import csv as pandas dataframe
production_file= pd.read_csv(path + "\\" + name_of_file)
production_file.head(3)

,name_adm2,production_tonnes
0,Abomey,15857
1,Abomey-Calavi,120957
2,Adja-ouere,118280


In [98]:
#Step2: Import the csv file containing the harvested area in level 2 according to the FAO DATA. This is the test output from the 
#: #downscaling process
path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data\Crop_Maps_modified_FAO"
name_of_file = "Ben_Cassava_test.csv"
# Import csv as pandas dataframe
input= pd.read_csv(path + "\\" + name_of_file)
input.head(3)

,Unnamed: 0,COUNTRY_CODE,COUNTRY_NAME,AREA_CODE,AREA_NAME,ADMIN_LEVEL,ITEM_CODE,ITEM_NAME,YEAR,AEREA_HARVESTED,YIELD,PRODUCTION,Admin_2_share,Area_Harv_2018
0,31,BEN,Benin,BEN012001,Abomey,2,125,Cassava,2000,509.0,7.000,3563.0,0.002510,704.836036
1,342,BEN,Benin,BEN003001,Abomey-Calavi,2,125,Cassava,2000,8568.0,6.858,58761.0,0.042251,11864.509143
2,613,BEN,Benin,BEN010001,Adjara,2,125,Cassava,2000,399.0,7.293,2910.0,0.001968,552.513906


In [99]:
input.columns

Index(['Unnamed: 0', 'COUNTRY_CODE', 'COUNTRY_NAME', 'AREA_CODE', 'AREA_NAME',
       'ADMIN_LEVEL', 'ITEM_CODE', 'ITEM_NAME', 'YEAR', 'AEREA_HARVESTED',
       'YIELD', 'PRODUCTION', 'Admin_2_share', 'Area_Harv_2018'],
      dtype='object')

In [100]:
# STEP3 : Drop all columns except the crop harvest area in 2018  and the district names colums and create a new dataframe
new= input.drop(['Unnamed: 0', 'COUNTRY_CODE', 'COUNTRY_NAME', 'AREA_CODE', 'ADMIN_LEVEL', 'ITEM_CODE',
                   'ITEM_NAME', 'YEAR', 'AEREA_HARVESTED','YIELD','PRODUCTION','Admin_2_share'],axis = 1)

In [101]:
# STEP4: Calculate the sum of the harvested area in each district (admin level 2)
new= new.groupby('AREA_NAME').sum()

In [102]:
# STEP 5: rename the column as required 
new.rename(columns={'Area_Harv_2018':'Harvest_Area_By_District'}, inplace=True)

In [103]:
#STEP6: merge the new dataframe to the initial input file
new_input = new.merge(input,on = 'AREA_NAME')

In [104]:
production_file.rename(columns={'name_adm2': 'AREA_NAME'}, inplace=True)

In [105]:
production_file.describe()

,production_tonnes
count,62.000000
mean,50685.290323
std,99032.191661
min,90.000000
25%,7347.250000
50%,23827.000000
75%,54540.500000
max,705797.000000


In [106]:
new_input.describe()


,Harvest_Area_By_District,Unnamed: 0,ADMIN_LEVEL,ITEM_CODE,YEAR,AEREA_HARVESTED,YIELD,PRODUCTION,Admin_2_share,Area_Harv_2018
count,72.000000,72.000000,72.0,72.0,72.0,72.000000,72.000000,72.000000,72.000000,72.000000
mean,3900.138889,10311.055556,2.0,125.0,2000.0,2816.500000,10.185986,29923.680556,0.013889,3900.138889
std,5554.706581,6098.586396,0.0,0.0,0.0,4011.352296,4.859656,45134.090286,0.019781,5554.706581
min,2.769493,31.000000,2.0,125.0,2000.0,2.000000,3.990000,10.000000,0.000010,2.769493
25%,707.605529,5136.000000,2.0,125.0,2000.0,511.000000,6.964500,3550.250000,0.002520,707.605529
50%,1922.720698,10289.500000,2.0,125.0,2000.0,1388.500000,8.924500,14792.000000,0.006847,1922.720698
75%,4226.592907,15498.500000,2.0,125.0,2000.0,3052.250000,12.717000,33040.000000,0.015051,4226.592907
max,32619.091662,20930.000000,2.0,125.0,2000.0,23556.000000,27.763000,248876.000000,0.116161,32619.091662


In [107]:
#STEP7: merge the production column (which is a seperate dataframe) with the output of the previous step 
new_input_final= new_input.merge(production_file,on ='AREA_NAME', how = 'left')

In [108]:
new_input_final.describe()

,Harvest_Area_By_District,Unnamed: 0,ADMIN_LEVEL,ITEM_CODE,YEAR,AEREA_HARVESTED,YIELD,PRODUCTION,Admin_2_share,Area_Harv_2018,production_tonnes
count,72.000000,72.000000,72.0,72.0,72.0,72.000000,72.000000,72.000000,72.000000,72.000000,56.000000
mean,3900.138889,10311.055556,2.0,125.0,2000.0,2816.500000,10.185986,29923.680556,0.013889,3900.138889,49790.035714
std,5554.706581,6098.586396,0.0,0.0,0.0,4011.352296,4.859656,45134.090286,0.019781,5554.706581,103477.838503
min,2.769493,31.000000,2.0,125.0,2000.0,2.000000,3.990000,10.000000,0.000010,2.769493,90.000000
25%,707.605529,5136.000000,2.0,125.0,2000.0,511.000000,6.964500,3550.250000,0.002520,707.605529,6971.250000
50%,1922.720698,10289.500000,2.0,125.0,2000.0,1388.500000,8.924500,14792.000000,0.006847,1922.720698,20178.000000
75%,4226.592907,15498.500000,2.0,125.0,2000.0,3052.250000,12.717000,33040.000000,0.015051,4226.592907,52556.250000
max,32619.091662,20930.000000,2.0,125.0,2000.0,23556.000000,27.763000,248876.000000,0.116161,32619.091662,705797.000000


In [109]:
new_input_final['ratio_harvested_area_by_district']= new_input_final['Area_Harv_2018']/new_input_final['Harvest_Area_By_District']

In [110]:
new_input_final['final_prod_tonnes']=new_input_final['ratio_harvested_area_by_district']*new_input_final['production_tonnes']

In [111]:
new_input_final

,AREA_NAME,Harvest_Area_By_District,Unnamed: 0,COUNTRY_CODE,COUNTRY_NAME,AREA_CODE,ADMIN_LEVEL,ITEM_CODE,ITEM_NAME,YEAR,AEREA_HARVESTED,YIELD,PRODUCTION,Admin_2_share,Area_Harv_2018,production_tonnes,ratio_harvested_area_by_district,final_prod_tonnes
0,Abomey,704.836036,31,BEN,Benin,BEN012001,2,125,Cassava,2000,509.0,7.000,3563.0,0.002510,704.836036,15857.0,1.0,15857.0
1,Abomey-Calavi,11864.509143,342,BEN,Benin,BEN003001,2,125,Cassava,2000,8568.0,6.858,58761.0,0.042251,11864.509143,120957.0,1.0,120957.0
2,Adja-ouere,11271.837584,9573,BEN,Benin,BEN011001,2,125,Cassava,2000,8140.0,15.082,122767.0,0.040140,11271.837584,118280.0,1.0,118280.0
3,Adjara,552.513906,613,BEN,Benin,BEN010001,2,125,Cassava,2000,399.0,7.293,2910.0,0.001968,552.513906,3771.0,1.0,3771.0
4,Adjohoun,2367.916741,884,BEN,Benin,BEN010002,2,125,Cassava,2000,1710.0,10.229,17491.0,0.008432,2367.916741,54582.0,1.0,54582.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Toucountouna,2077.119948,18913,BEN,Benin,BEN002009,2,125,Cassava,2000,1500.0,12.000,18000.0,0.007397,2077.119948,76351.0,1.0,76351.0
68,Toviklin,1467.831430,19745,BEN,Benin,BEN006006,2,125,Cassava,2000,1060.0,16.000,16960.0,0.005227,1467.831430,NaN,1.0,NaN
69,Za-kpota,1793.246888,20036,BEN,Benin,BEN012007,2,125,Cassava,2000,1295.0,6.111,7914.0,0.006386,1793.246888,NaN,1.0,NaN
70,Zangnanado,2689.177959,20347,BEN,Benin,BEN012008,2,125,Cassava,2000,1942.0,5.908,11474.0,0.009577,2689.177959,NaN,1.0,NaN


In [112]:
x=new_input_final['final_prod_tonnes'].sum()
print(x)

2788242.0


In [113]:
path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data\Crop_Maps_modified_FAO"

new_input_final.to_csv(os.path.join(path,"{c}.csv".format(c="Ben_Cassava_final")))